In [102]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_recall_fscore_support
from sklearn.feature_selection import SelectKBest, f_classif  # SelectKBest와 평가 지표 import

In [103]:
# 1. 데이터 불러오기
data = pd.read_csv('azdigar nav sartirovka+0 delete.csv')

In [104]:
# 2. 분석할 열들 정의
columns = [
    'w08chronic_a', 'w08chronic_b', 'w08chronic_c',
    'w08chronic_d', 'w08chronic_e', 'w08chronic_f',
    'w08chronic_g', 'w08chronic_h', 'w08chronic_i',
    'w08chronic_k', 'w08chronic_l', 'w08chronic_m'
]

In [105]:
# 3. 데이터 클리닝 - 'w08chronic_m'에서 값이 3인 행 제거
cleaned_data = data[data['w08chronic_m'] != 3]

In [106]:
# 4. 피처와 타겟 정의
X = cleaned_data.drop(columns=columns)
y_all = cleaned_data[columns]


In [109]:
# 5. SelectKBest를 통한 피처 선택
k = 5  # 선택할 피처 수
selected_features = []

for target in columns:
    selector = SelectKBest(score_func=f_classif, k=k)
    X_target_selected = selector.fit_transform(X, y_all[target])
    selected_features.append(X.columns[selector.get_support()].tolist())

# 6. 선택된 피처 확인
print("Selected Features for each target:\n", selected_features)

Selected Features for each target:
 [['w08A002y', 'w08A002_age', 'w08C001', 'w08C152', 'w08C343'], ['w08A002y', 'w08A002_age', 'w08C001', 'w08C152', 'w08C343'], ['w08C001', 'w08C152', 'w08C318', 'w08C343', 'w08G026'], ['w08A002y', 'w08A002_age', 'w08C001', 'w08C337', 'w08C343'], ['w08gender1', 'w08C001', 'w08C152', 'w08C003', 'w08C343'], ['w08A002y', 'w08A002_age', 'w08C001', 'w08C152', 'w08C343'], ['w08C152', 'w08C202', 'w08C209', 'w08C213', 'w08C343'], ['w08C152', 'w08Cadd_19', 'w08C145', 'w08C148', 'w08C301'], ['w08edu', 'w08gender1', 'w08A002y', 'w08A002_age', 'w08mgrip'], ['w08C001', 'w08C081', 'w08C105', 'w08C334', 'w08C343'], ['w08C152', 'w08C002', 'w08C318', 'w08C334', 'w08C337'], ['w08C203', 'w08C208', 'w08C214', 'w08C216', 'w08C217']]


c:\Users\azamj\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [ 86 120 121 122] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\azamj\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
c:\Users\azamj\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [ 86 120 121 122] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\azamj\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
c:\Users\azamj\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [ 86 120 121 122] are constant.
  warnings.warn("Features %s are constant."

In [111]:
# 7. 선택된 피처 통합
selected_features_flat = list(set(feature for sublist in selected_features for feature in sublist))
X_selected = X[selected_features_flat]


In [112]:
# 8. 데이터 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected)

In [113]:
# 9. 학습 및 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_all, test_size=0.2, random_state=42)

In [114]:
# 10. 모델 정의 및 학습
rf = RandomForestClassifier()
multi_target_rf = MultiOutputClassifier(rf, n_jobs=-1)
multi_target_rf.fit(X_train, y_train)

MultiOutputClassifier(estimator=RandomForestClassifier(), n_jobs=-1)

In [115]:
# 11. 모델 예측
y_pred = multi_target_rf.predict(X_test)

In [122]:
y_test_dense = y_test.to_numpy() if hasattr(y_test, 'to_numpy') else y_test


In [125]:
print(type(y_test_dense))
print(type(y_pred_dense))


<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>


In [128]:
from sklearn.metrics import precision_recall_fscore_support
import pandas as pd

# y_test_dense가 sparse matrix인 경우, 밀집 배열로 변환
if hasattr(y_test_dense, 'toarray'):
    y_test_dense = y_test_dense.toarray()
elif isinstance(y_test_dense, pd.DataFrame):
    y_test_dense = y_test_dense.values

# y_pred_dense가 sparse matrix인 경우, 밀집 배열로 변환
if hasattr(y_pred_dense, 'toarray'):
    y_pred_dense = y_pred_dense.toarray()
elif isinstance(y_pred_dense, pd.DataFrame):
    y_pred_dense = y_pred_dense.values

# 결과 평가
results = precision_recall_fscore_support(y_test_dense, y_pred_dense, average=None)
results_df = pd.DataFrame(results, index=columns).T
results_df.columns = ['Precision', 'Recall', 'F1-Score', 'Support']

# 결과 출력
print(results_df)


ValueError: Unknown label type: (<906x12 sparse matrix of type '<class 'numpy.int64'>'
	with 10872 stored elements in Compressed Sparse Row format>, <906x12 sparse matrix of type '<class 'numpy.int64'>'
	with 10872 stored elements in Compressed Sparse Row format>)